In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor
from xgboost import to_graphviz, plot_importance

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import _hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor, HistGradientBoostingRegressor
%matplotlib inline

from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report, mean_absolute_error, mean_squared_error,r2_score
from sklearn.metrics import plot_confusion_matrix, plot_precision_recall_curve, plot_roc_curve

import keras
from keras.models import save_model, Sequential
from keras.layers import Activation, BatchNormalization, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

pd.options.display.max_columns= None
#pd.options.display.max_rows = None

Using TensorFlow backend.


In [2]:
df = pd.read_csv("train2.csv",low_memory=False)

In [3]:
df

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,change_No,diabetesMed_Yes,readmitted
0,-1.137649,-0.106517,-0.785398,-1.848268,-0.291461,-0.21262,-0.503276,-3.321596,0.926591,-1.829868,0
1,1.877333,0.655900,0.387074,0.981622,-0.291461,-0.21262,-0.503276,0.815784,-1.079225,0.546488,1
2,1.542335,0.910040,1.559545,1.104660,-0.291461,-0.21262,-0.503276,0.815784,-1.079225,0.546488,0
3,2.212331,1.265834,-0.785398,1.104660,-0.291461,-0.21262,-0.503276,0.815784,-1.079225,0.546488,1
4,0.537341,0.198450,0.387074,-0.371804,-0.291461,-0.21262,-0.503276,0.298612,0.926591,-1.829868,0
...,...,...,...,...,...,...,...,...,...,...,...
101761,-1.137649,-2.139630,0.387074,-1.356113,-0.291461,-0.21262,-0.503276,-0.218561,0.926591,0.546488,0
101762,-0.132655,1.164179,-0.785398,-0.002688,-0.291461,-0.21262,-0.503276,0.815784,0.926591,-1.829868,0
101763,0.537341,0.910040,-0.199162,-0.248765,-0.291461,-0.21262,-0.503276,0.815784,0.926591,0.546488,0
101764,1.542335,0.096794,2.732016,0.858583,-0.291461,-0.21262,-0.503276,0.815784,0.926591,-1.829868,0


In [4]:
df.shape

(101766, 11)

### Train a Neural Model

In [5]:
X = df.iloc[:,0:10]
y = df.iloc[:,10]

In [6]:
X.values, y.values

(array([[-1.13764856, -0.10651686, -0.78539774, ..., -3.3215959 ,
          0.92659052, -1.82986799],
        [ 1.87733278,  0.65590042,  0.38707362, ...,  0.81578448,
         -1.07922537,  0.54648751],
        [ 1.54233485,  0.91003951,  1.55954498, ...,  0.81578448,
         -1.07922537,  0.54648751],
        ...,
        [ 0.53734107,  0.91003951, -0.19916206, ...,  0.81578448,
          0.92659052,  0.54648751],
        [ 1.54233485,  0.09679441,  2.73201634, ...,  0.81578448,
          0.92659052, -1.82986799],
        [-0.46765271,  1.06252297, -0.19916206, ...,  0.81578448,
         -1.07922537,  0.54648751]]),
 array([0, 1, 0, ..., 0, 0, 0], dtype=int64))

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [8]:
X_train.shape, X_test.shape, y_train.shape

((81412, 10), (20354, 10), (81412,))

In [9]:
model = Sequential()

model.add(Dense(32,activation='relu',input_shape=(10,)))
model.add(Dropout(0.2))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))


In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                352       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                2112      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                

In [11]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [12]:
cb = ModelCheckpoint(filepath='lab5a.h5',verbose=2,save_best_only=True)

In [13]:
es = EarlyStopping(patience=20,verbose=2, restore_best_weights=True)

In [14]:
model.fit(X_train,y_train,batch_size=500,epochs=100,verbose=2,validation_split=0.2,
          validation_data=(X_test,y_test),callbacks=[cb,es])

Train on 81412 samples, validate on 20354 samples
Epoch 1/100
 - 2s - loss: 0.6663 - accuracy: 0.5947 - val_loss: 0.6541 - val_accuracy: 0.6141

Epoch 00001: val_loss improved from inf to 0.65411, saving model to lab5a.h5
Epoch 2/100
 - 1s - loss: 0.6572 - accuracy: 0.6124 - val_loss: 0.6524 - val_accuracy: 0.6165

Epoch 00002: val_loss improved from 0.65411 to 0.65236, saving model to lab5a.h5
Epoch 3/100
 - 1s - loss: 0.6543 - accuracy: 0.6157 - val_loss: 0.6532 - val_accuracy: 0.6183

Epoch 00003: val_loss did not improve from 0.65236
Epoch 4/100
 - 1s - loss: 0.6533 - accuracy: 0.6184 - val_loss: 0.6510 - val_accuracy: 0.6202

Epoch 00004: val_loss improved from 0.65236 to 0.65100, saving model to lab5a.h5
Epoch 5/100
 - 1s - loss: 0.6521 - accuracy: 0.6197 - val_loss: 0.6517 - val_accuracy: 0.6192

Epoch 00005: val_loss did not improve from 0.65100
Epoch 6/100
 - 1s - loss: 0.6521 - accuracy: 0.6204 - val_loss: 0.6516 - val_accuracy: 0.6206

Epoch 00006: val_loss did not improve f

 - 1s - loss: 0.6476 - accuracy: 0.6252 - val_loss: 0.6522 - val_accuracy: 0.6189

Epoch 00055: val_loss did not improve from 0.64890
Epoch 56/100
 - 1s - loss: 0.6476 - accuracy: 0.6250 - val_loss: 0.6500 - val_accuracy: 0.6211

Epoch 00056: val_loss did not improve from 0.64890
Epoch 57/100
 - 1s - loss: 0.6474 - accuracy: 0.6255 - val_loss: 0.6508 - val_accuracy: 0.6205

Epoch 00057: val_loss did not improve from 0.64890
Epoch 58/100
 - 1s - loss: 0.6475 - accuracy: 0.6251 - val_loss: 0.6497 - val_accuracy: 0.6214

Epoch 00058: val_loss did not improve from 0.64890
Restoring model weights from the end of the best epoch
Epoch 00058: early stopping


In [15]:
scores = model.evaluate(X,y)

101766/101766 [==============================] - 4s 35us/step


In [16]:
scores  #62.54% accuracy

[0.6462463303430227, 0.6254348158836365]

In [17]:
y_pred = model.predict(X_test)

In [18]:
y_pred

array([[0.5858819 ],
       [0.33099866],
       [0.6874957 ],
       ...,
       [0.43394154],
       [0.32295805],
       [0.48286757]], dtype=float32)

In [19]:
y_pred = model.predict_classes(X_test)

In [20]:
y_pred

array([[1],
       [0],
       [1],
       ...,
       [0],
       [0],
       [0]])

In [23]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.62      0.76      0.68     10993
           1       0.62      0.46      0.53      9361

    accuracy                           0.62     20354
   macro avg       0.62      0.61      0.61     20354
weighted avg       0.62      0.62      0.61     20354

